## To run this model, go to the top left bar -> Run -> Run All Cells

### Model runtime: approx. 10-15min

In [1]:
from epx import Job, ModelConfig, SynthPop

import json
import pandas as pd
import geopandas as gpd
import numpy as np
import time

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as splt
import requests

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

r = requests.get("https://gist.githubusercontent.com/daniel-epistemix/8009ad31ebfa96ac97b7be038c014c0d/raw/320c3b0ca3dfbf7946e49c97254fa65d4753aeac/epx_plotly_theme.json")
if r.status_code == 200:
    pio.templates["epistemix"] = go.layout.Template(r.json())
    pio.templates.default = "epistemix"

mapstyle="mapbox://styles/epxadmin/cm0ve9m13000501nq8q1zdf5p"
token="pk.eyJ1IjoiZXB4YWRtaW4iLCJhIjoiY20wcmV1azZ6MDhvcTJwcTY2YXpscWsxMSJ9._ROunfMS6hgVh1LPQZ4NGg"

In [ ]:
asthma_config = ModelConfig(
             synth_pop=SynthPop("US_2010.v5", ["Hennepin_County_MN"]),
             start_date="2024-01-01",
             end_date="2024-12-30",
         )

asthma_job = Job(
    "model/main.fred",
    config=[asthma_config],
    key="cl_asthma_job",
    fred_version="11.1.1",
    results_dir="/home/epx/cl-results"
)


asthma_job.execute()

# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 720 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(asthma_job.status) != 'DONE':
    if str(asthma_job.status) == 'ERROR':
        logs = asthma_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")
str(asthma_job.status)

SSLError: HTTPSConnectionPool(host='studio.epistemix.cloud', port=443): Max retries exceeded with url: /v1/runs (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1002)')))

In [ ]:
## Reading in relevant inputs (daily AQI, hospital locations)

aqi_msp = pd.read_csv("data/aqidaily2023_minneapolis.csv")

hospitals = pd.read_csv("data/hospitals_MN.csv")
hospitals["agent_type"] = 2
hospitals[["ID", "agent_type", "lat", "lon"]].to_csv("data/input_hospitals.csv", index=False)

hosp_df = gpd.GeoDataFrame(hospitals, 
                 geometry=gpd.points_from_xy(hospitals.lon, hospitals.lat))


## and outputs (agent asthma status, records of asthma events)

msp_agent_info = asthma_job.results.csv_output("agent_info.csv")

msp_asthma_events = asthma_job.results.csv_output("asthma_event.csv")
msp_asthma_events["date"] = pd.to_datetime(msp_asthma_events.day.astype(str), format='%Y%m%d')

msp_event_df = msp_asthma_events.merge(msp_agent_info, on="id")
msp_event_df = msp_event_df.merge(hospitals[["ID", "NAME"]].rename(columns={"ID":"hosp", "NAME":"hospital"}), on="hosp")

In [ ]:
## Calculating asthma prevalence across the population as a function of age and race
## Race is represented by a numeric, with 1 : white and 2 : black

total_y, total_x = np.histogram(msp_agent_info.age,bins=np.arange(0,101,5))
asthma_y, asthma_x = np.histogram(msp_agent_info[msp_agent_info.status==1].age,bins=np.arange(0,101,5))

total_w_y, total_w_x = np.histogram(msp_agent_info[msp_agent_info.race==1].age,bins=np.arange(0,101,5))
asthma_w_y, asthma_w_x = np.histogram(msp_agent_info[(msp_agent_info.race==1)&(msp_agent_info.status==1)].age,bins=np.arange(0,101,5))

total_b_y, total_b_x = np.histogram(msp_agent_info[msp_agent_info.race==2].age,bins=np.arange(0,101,5))
asthma_b_y, asthma_b_x = np.histogram(msp_agent_info[(msp_agent_info.race==2)&(msp_agent_info.status==1)].age,bins=np.arange(0,101,5))

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=pd.to_datetime(aqi_msp["Date"])+pd.DateOffset(365),
        y=aqi_msp["Overall AQI Value"],
        mode="lines",
        line=go.scatter.Line(width=2),
        showlegend=True,
        name="Minneapolis")
)

fig.update_layout(
    font_family="Epistemix Label",
    legend_title="AQI",
    yaxis_title="Overall AQI",
    xaxis_title="Date",
    title="Projected Daily AQI, 2024",
    title_font_size=24,
    #xaxis_range=["2022-01-01","2022-04-01"],
    hovermode="x unified",height=400,
)

fig.show()

asthma_status_fig = go.Figure()

asthma_status_fig.add_trace(
    go.Scatter(x=total_x[1:], y=100*asthma_y/total_y, 
               mode="lines",line=go.scatter.Line(width=4),
               name="Total Incidence %")
)

asthma_status_fig.add_trace(
    go.Bar(x=total_w_x[1:], y=100*asthma_w_y/total_w_y,
           name="% of White Population")
)

asthma_status_fig.add_trace(
    go.Bar(x=total_b_x[1:], y=100*asthma_b_y/total_b_y,
           name="% of Black Population")
)

asthma_status_fig.update_layout(
    font_family="Epistemix Label",
    yaxis_title="Population %",
    xaxis_title="Age",
    title="Prevalence of Asthma by Age and Race",
    title_font_size=24,
    #xaxis_range=["2022-01-01","2022-04-01"],
    hovermode="x unified",height=400,
)

asthma_status_fig.show()

In [ ]:
fig1 = px.scatter_mapbox(msp_agent_info[msp_agent_info.insurance<0.8].sample(20000), 
                        lat="hh_lat", 
                        lon="hh_lon",     
                        opacity=0.6,
                        size_max=4,
                        size="risk",
                        color_discrete_sequence=["#F3F3F7"],
                        zoom=9.8,#height=1080,width=1920
                        height=700,
                       )

dict1 = fig1.to_dict()["data"][0]


fig2 = px.scatter_mapbox(msp_agent_info.sample(2000), 
                        lat="hh_lat", 
                        lon="hh_lon",     
                        opacity=0.8,
                        color="risk",
                        color_continuous_scale="plasma",
                        zoom=9.5,#height=1080,width=1920
                        height=700,
                        hover_data=["id", "age", "race"]
                       )

dict2 = fig2.to_dict()["data"][0]

layout_dict = fig2.to_dict()["layout"]


In [ ]:
map_fig = go.Figure()

map_fig.add_trace(
    go.Scattermapbox(lat=dict1["lat"], lon=dict1["lon"], 
                     marker={'color':"#8C96CA", 
                             'opacity':1, 'size':4},
                     name="Non-United Members", hoverinfo='skip'), 
)

map_fig.update_layout(mapbox=layout_dict["mapbox"],
                      mapbox_style=mapstyle, mapbox_accesstoken=token,
                  margin={"r":0,"t":60,"l":0,"b":0}, height=500,
                      title="Subsample of Synthetic Agents; Minneapolis MN",
                  title_font_size=24,showlegend=False)
map_fig.show()

In [ ]:
map_fig = go.Figure()

map_fig.add_trace(
    go.Scattermapbox(lat=dict1["lat"], lon=dict1["lon"], 
                     marker={'color':"#8C96CA", 
                             'opacity':1, 'size':4},
                     name="Non-United Members", hoverinfo='skip'), 
)

map_fig.add_trace(
    go.Scattermapbox(
        lat=dict2["lat"], lon=dict2["lon"], 
        marker={'color':dict2["marker"]["color"], 
                'opacity':1, 'size':9,'coloraxis':'coloraxis'}, 
        name="United Members",
        customdata=dict2["customdata"],
        hovertemplate=dict2["hovertemplate"]
    ),
)

map_fig.update_layout(mapbox=layout_dict["mapbox"],
                      mapbox_style=mapstyle, mapbox_accesstoken=token,
                  margin={"r":0,"t":60,"l":0,"b":0}, height=700,
                  title="Synthesized Asthma Risk Across Minneapolis",
                  title_font_size=24,showlegend=True, 
                      coloraxis={"colorbar":{"title":{"text":"Asthma Risk"}, "len":0.8, "lenmode":"fraction"},
                                 "colorscale":layout_dict["coloraxis"]["colorscale"]})
map_fig.show()

tmp_ins = []
for i in range(len(msp_event_df)):
    x = msp_event_df.iloc[i]
    if (x.age >= 18) and (x.age<65):
        tmp_ins += [x.insurance+0.2]
    else:
        tmp_ins += [x.insurance]

msp_event_df["tmp_ins"] = tmp_ins

msp_event_df["insurance_threshold"] = 0
msp_event_df.loc[msp_event_df['tmp_ins'] > 0.8, "insurance_threshold"] = 1

fig = go.Figure()

fig.add_trace(go.Histogram(x=msp_event_df["age"],
                           nbinsx=25, name="Total Population"))
fig.add_trace(go.Histogram(x=msp_event_df[msp_event_df.insurance_threshold==1]["age"],
                           nbinsx=25, name="United Members"))

fig.update_layout(xaxis_title="Age", yaxis_title="Asthma Hospitals, 2024", 
                  title="Asthma Hospitalizations Across Age", title_font_size=24,
                  hovermode="x unified")


In [ ]:
msp_event_df["cost"] = np.random.normal(loc=5000, scale=1000, size=len(msp_event_df))

fig = splt.make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Histogram(x=msp_event_df["date"], y=msp_event_df["cost"], histfunc="sum",
                 name="Asthma Events",)
)

fig.add_trace(go.Scatter(x=pd.to_datetime(aqi_msp.Date)+pd.DateOffset(365), 
                         y=aqi_msp["Overall AQI Value"], 
                         name="MSP AQI"), 
    secondary_y=True)


fig.update_layout(
    title="Hospital visits as a function of environment",
    title_font_size=24,
    yaxis=dict(
        title="Total Weekly Cost",
        titlefont=dict(
            color="#6383F3"
        ),
        tickfont=dict(
            color="#6383F3"
        )
    ),
    yaxis2=dict(
        title="AQI",
        titlefont=dict(
            color="#EB5A36"
        ),
        tickfont=dict(
            color="#EB5A36"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=0.95
    )
)
fig.update_layout(hovermode="x unified")
fig.show()

In [ ]:
fig = px.scatter_mapbox(msp_event_df,
                        lat="hh_lat", 
                        lon="hh_lon",     
                        opacity=0.8,
                        color="hospital",
                        zoom=9,#height=1080,width=1920
                        height=500,
                        hover_data=["id", "age", "race"]
                       )

fig.update(layout_showlegend=False)
fig.update_layout(mapbox_style=mapstyle, mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":60,"l":0,"b":0})
#fig.update_layout(font=dict(size=22))
fig.update_layout(title="Acute Asthma Events by Hospital",title_font_size=24,)

msp_event_hosp = msp_event_df.groupby(["date","hospital"]).id.count().to_frame().reset_index().rename(columns={"id":"event_count"})

fig1 = px.histogram(msp_event_hosp, y="date", x="event_count", 
                   color="hospital", barmode="group", nbins=52,
                   orientation="h", height=500,)
#fig.update(layout_showlegend=False)
fig1.update_yaxes(range=["2024-06-28", "2024-06-01"])
fig1.update_layout(xaxis_title="Weekly Asthma Events per Hospital", )

fig.show()
fig1.show()


In [ ]:
asthma_job.delete(interactive=False)